In [3]:
# from huggingface_hub import login
# login()  # Hugging Face 토큰을 입력하라는 프롬프트가 표시됨

In [2]:
import torch
torch.cuda.empty_cache()

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)


# Load the tokenizer and model with 4-bit quantization
# model_name = "deepseek-ai/deepseek-llm-7b-base"
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# skpark: pad_token을 eos_token으로 설정
tokenizer.pad_token = tokenizer.eos_token





host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0


In [4]:
# model.device

## Dataset Preparation

In [5]:
# import pandas as pd

# splits = {'train': 'synthetic_text_to_sql_train.snappy.parquet', 'test': 'synthetic_text_to_sql_test.snappy.parquet'}
# df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["train"])

In [2]:
from datasets import load_dataset

# # Load the Spider dataset
# spider_dataset = load_dataset("spider")


# Load the Synthetic Text-to-SQL dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql")

host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  s3.amazonaws.com
port:  443
family:  0
host:  datasets-server.huggingface.co
port:  443
family:  0
host:  datasets-server.huggingface.co
port:  443
family:  0
host:  datasets-server.huggingface.co
port:  443
family:  0
host:  datasets-server.huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:

In [3]:
dataset['train']

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 100000
})

In [4]:
# dataset['train'].filter(lambda x: x["domain"] in ['technology', 'telecommunications'])
dataset['train'] = dataset['train'].filter(lambda x: x["domain"] in ['fashion retail', 'retail'])


In [5]:
dataset['test'] = dataset['test'].filter(lambda x: x["domain"] in ['fashion retail', 'retail'])

In [6]:


# # Preprocessing function
# def preprocess_function(examples):
#     inputs = [f"Question: {q} SQL: " for q in examples["question"]]
#     targets = examples["query"]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs



# # Apply preprocessing
# tokenized_dataset = spider_dataset.map(preprocess_function, batched=True)



In [7]:


# Preprocessing function
def preprocess_function(examples):
    # inputs = [f"Question: {q} SQL: " for q in examples["question"]]
    inputs = [f"Question: {question} \n\n Schema: {schema} \n\n SQL query: " 
              for question, schema in zip(examples["sql_prompt"], examples["sql_context"])]
    # targets = examples["query"]
    targets = examples["sql"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/104 [00:00<?, ? examples/s]

/home/sprk/workspace/envs/env_sft/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
tokenized_dataset['train'][0]['labels']

[128000,
 4963,
 320,
 39243,
 29771,
 353,
 220,
 1041,
 13,
 15,
 611,
 320,
 4963,
 24945,
 29771,
 4393,
 3956,
 595,
 439,
 11668,
 4393,
 3956,
 5401,
 374,
 7225,
 37676,
 284,
 8378,
 26,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 128001,
 12

In [9]:
tokenized_dataset['train'][0]

{'id': 5145,
 'domain': 'retail',
 'domain_description': 'Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.',
 'sql_complexity': 'subqueries',
 'sql_complexity_description': 'subqueries, including correlated and nested subqueries',
 'sql_task_type': 'analytics and reporting',
 'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
 'sql_prompt': 'What is the percentage of products that are made from recycled materials?',
 'sql_context': 'CREATE TABLE products(product_id INT, is_recycled BOOLEAN);',
 'sql': 'SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products)) as percentage FROM products WHERE is_recycled = TRUE;',
 'sql_explanation': 'This query calculates the percentage of products that are made from recycled materials by dividing the count of recycled products by the total number of products.',
 'input_ids': [128000,
  14924,
  25,
  3639,
  374,
  279,
  11668,
  315,
  3956,
  430,
 

In [24]:
import bitsandbytes
from peft import LoraConfig, PeftModel  # Import LoraConfig from peft



# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.float16
)


# # FP16 양자화 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=False,
#     load_in_8bit=False,
#     # bnb_4bit_quant_type=None,
#     bnb_4bit_compute_dtype=torch.float16  # FP16 사용
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # device_map="auto"
    # device_map="cpu"
)

#skpark
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)

# Define LoRA configuration
lora_config = LoraConfig(
    # r=8,  # Rank of the LoRA update matrices,  7B~10B 모델에 적합
    r=16,  # Rank of the LoRA update matrices,  7B 미만 모델에 적합
    # lora_alpha=32,  # Scaling factor for the LoRA updates
    # lora_alpha=16,  # Scaling factor for the LoRA updates
    lora_alpha=32,  # Scaling factor for the LoRA updates
    target_modules=["q_proj", "k_proj", "v_proj"],  # Specific target modules for LLM
    # lora_dropout=0.05,  # Dropout probability for the LoRA layers
    lora_dropout=0.02,  # Dropout probability for the LoRA layers
    bias="none",  # No bias for the LoRA layers
)


# Apply LoRA
# The model is already quantized during loading using bnb_config, and compute dtype is set to bfloat16.)
from peft import get_peft_model # Import get_peft_model
model = get_peft_model(model, lora_config) # Use get_peft_model to add LoRA to the model

# Configure the model for training
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

cuda


In [25]:

# EarlyStoppingCallback 설정
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # 검증 손실이 개선되지 않는 에포크 수
    early_stopping_threshold=0.01  # 개선으로 간주되는 최소 변화량
)



# Define training arguments
training_args = TrainingArguments(
    output_dir="./results/synthetic_text_to_sql",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    # learning_rate=2e-5,
    learning_rate=4e-5,
    # num_train_epochs=1,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    # logging_steps=10,
    # save_steps=500,
    logging_steps=10,
    logging_dir='./logs',
    save_steps=100,
    # evaluation_strategy="epoch",
    push_to_hub=False,
    # push_to_hub=True,
    # hub_model_id="frankmorales2020/deepseek-llm-7b-base-spider",
    report_to="none"
)







# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    # eval_dataset=tokenized_dataset["validation"],
)



No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [26]:
# Fine-tune the model
trainer.train()

# Save and push the fine-tuned model to Hugging Face Hub
trainer.save_model()
# trainer.push_to_hub()

Step,Training Loss
10,12.615000
20,6.827300
30,2.511600
40,1.482300
50,1.067000
60,0.809700
70,0.750700
80,0.722100
90,0.673700
100,0.609400


host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443
family:  0
host:  huggingface.co
port:  443

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 원본 모델 로드
base_model_name = model_name  # 원본 모델 이름
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# LoRA 어댑터 로드 및 결합
lora_adapter_path = "/home/sprk/workspace/project/text2sql-finetuning/results/synthetic_text_to_sql/adapter_model.safetensors"
model = PeftModel.from_pretrained(base_model, lora_adapter_path)

# 추론 모드로 설정
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at '/home/sprk/workspace/project/text2sql-finetuning/results/synthetic_text_to_sql/checkpoint-590/adapter_model.safetensors'

In [ ]:
text = "An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)


In [ ]:
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

In [ ]:

# 평가를 위한 TrainingArguments 설정
eval_args = TrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=4,
    do_predict=True,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_dataset["test"],
)

# 모델 평가
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")